# Setup

In [2]:
from langchain_groq import ChatGroq

In [3]:
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader


In [4]:
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

True

In [5]:
groq = ChatGroq(model_name='llama-3.3-70b-versatile',temperature=0)

# SequentialChain

In [6]:
# Carregar os documentos
loader = TextLoader('base_conhecimento_britadeira.txt')
documents = loader.load()
# Carregar histórico de conversas 
historico_conversas = """Cliente: Minha britadeira não liga. Chatbot: Você já verificou 
                         se a bateria está carregada e conectada corretamente?"""
# Pergunta do cliente
pergunta = "Minha britadeira não liga. Eu já veriquei e a bateria está carregada e conectada corretamente"

In [7]:
inputs = {
    "context": "\n".join(doc.page_content for doc in documents),
    "question": pergunta,
    "historico": historico_conversas
}

In [8]:
prompt_base_conhecimento = PromptTemplate(
    input_variables=["context", "question"],
    template="""Use o seguinte contexto para responder à pergunta. 
    Responda apenas com base nas informações fornecidas.
    Não forneceça instruções de procedimento já realizados.
    Não utilize informações externas ao contexto:
    Contexto: {context}
    Pergunta: {question}"""
)


In [9]:
prompt_historico_conversas = PromptTemplate(
    input_variables=["historico", "question"],
    template="""Use o histórico de conversas para responder à pergunta. 
    Responda apenas com base nas informações fornecidas. 
    Não forneceça instruções de procedimento já realizados.
    Não utilize informações externas ao contexto:
    Histórico: {historico}
    Pergunta: {question}"""
)


In [10]:
prompt_final = PromptTemplate(
    input_variables=["resposta_base_conhecimento", "resposta_historico_conversas"],
    template="""Combine as seguintes respostas para gerar uma resposta final,
    mas não forneça instruções de procedimentos já realizados:
    Resposta da base de conhecimento: {resposta_base_conhecimento}
    Resposta do histórico de conversas: {resposta_historico_conversas}"""
)

In [11]:
print(prompt_base_conhecimento)

input_variables=['context', 'question'] input_types={} partial_variables={} template='Use o seguinte contexto para responder à pergunta. \n    Responda apenas com base nas informações fornecidas.\n    Não forneceça instruções de procedimento já realizados.\n    Não utilize informações externas ao contexto:\n    Contexto: {context}\n    Pergunta: {question}'


In [12]:
# Definir as cadeias  
chain_base_conhecimento = prompt_base_conhecimento | groq
chain_historico_conversas = prompt_historico_conversas | groq
chain_final = prompt_final | groq

In [13]:
print(chain_base_conhecimento)

first=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use o seguinte contexto para responder à pergunta. \n    Responda apenas com base nas informações fornecidas.\n    Não forneceça instruções de procedimento já realizados.\n    Não utilize informações externas ao contexto:\n    Contexto: {context}\n    Pergunta: {question}') middle=[] last=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x75efd4fd67b0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x75efd4fd7380>, model_name='llama-3.3-70b-versatile', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))


In [14]:
print(chain_historico_conversas)

first=PromptTemplate(input_variables=['historico', 'question'], input_types={}, partial_variables={}, template='Use o histórico de conversas para responder à pergunta. \n    Responda apenas com base nas informações fornecidas. \n    Não forneceça instruções de procedimento já realizados.\n    Não utilize informações externas ao contexto:\n    Histórico: {historico}\n    Pergunta: {question}') middle=[] last=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x75efd4fd67b0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x75efd4fd7380>, model_name='llama-3.3-70b-versatile', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))


In [15]:
# Passando dados e executando
resultado_base_conhecimento = chain_base_conhecimento.invoke({"context": inputs["context"], "question": inputs["question"]})
resultado_historico_conversas = chain_historico_conversas.invoke({"historico": inputs["historico"], "question": inputs["question"]})
resultado_final = chain_final.invoke({"resposta_base_conhecimento": resultado_base_conhecimento, 
                                      "resposta_historico_conversas": resultado_historico_conversas})

In [16]:
print("Resultado Base de Conhecimento:\n", resultado_base_conhecimento)
print("----")
print("Resultado Histórico de Conversas:\n", resultado_historico_conversas)

Resultado Base de Conhecimento:
 content='Aguarde 30 segundos após conectar a bateria para que o sistema inicialize. Além disso, verifique se o interruptor liga/desliga está na posição correta. Se o problema persistir, contate o suporte técnico pelo 0800 555 5555.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 831, 'total_tokens': 895, 'completion_time': 0.229649641, 'prompt_time': 0.070646315, 'queue_time': 0.269751792, 'total_time': 0.300295956}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--50450cc7-b77a-4210-94eb-fa6a901b5cc1-0' usage_metadata={'input_tokens': 831, 'output_tokens': 64, 'total_tokens': 895}
----
Resultado Histórico de Conversas:
 content='Verifique se o cabo ou a fonte de alimentação está funcionando corretamente e se não há danos visíveis na britadeira. Além disso, certifique-se de que o interruptor o

In [17]:
print(resultado_final.content)

Para resolver o problema com a britadeira, é importante seguir alguns passos. Primeiramente, aguarde 30 segundos após conectar a bateria para que o sistema inicialize. Além disso, verifique se o interruptor liga/desliga está na posição correta. 

É fundamental também verificar se o cabo ou a fonte de alimentação está funcionando corretamente e se não há danos visíveis na britadeira. Certifique-se de que o interruptor ou gatilho esteja funcionando corretamente. Se o problema persistir, pode ser necessário verificar se há algum problema interno na britadeira, como um problema com o motor ou outros componentes internos. 

Se mesmo após essas verificações o problema não for resolvido, é recomendável contatar o suporte técnico pelo 0800 555 5555 para obter assistência especializada.


In [18]:
print(resultado_final)

content='Para resolver o problema com a britadeira, é importante seguir alguns passos. Primeiramente, aguarde 30 segundos após conectar a bateria para que o sistema inicialize. Além disso, verifique se o interruptor liga/desliga está na posição correta. \n\nÉ fundamental também verificar se o cabo ou a fonte de alimentação está funcionando corretamente e se não há danos visíveis na britadeira. Certifique-se de que o interruptor ou gatilho esteja funcionando corretamente. Se o problema persistir, pode ser necessário verificar se há algum problema interno na britadeira, como um problema com o motor ou outros componentes internos. \n\nSe mesmo após essas verificações o problema não for resolvido, é recomendável contatar o suporte técnico pelo 0800 555 5555 para obter assistência especializada.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 194, 'prompt_tokens': 625, 'total_tokens': 819, 'completion_time': 0.389683628, 'prompt_time': 0.119016311, 'queue_time'